## Load DataSet

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from keras.datasets import cifar100
from keras.utils import to_categorical
import numpy as np
import cv2
import os
from keras import backend as K

In [ ]:
(x_train, y_train_), (x_test, y_test_) = cifar100.load_data()


169009152/169001437 [==============================] - 6s 0us/step


In [ ]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [ ]:
y_train = to_categorical(y_train_)
y_test = to_categorical(y_test_)

## ResNet18 Model

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Conv2D, Input
from keras.layers import MaxPooling2D, AveragePooling2D, Dropout
from keras.callbacks import EarlyStopping
from keras.layers.merge import concatenate
from keras.regularizers import l2
import os

In [ ]:
input_shape = (32, 32, 3)
input = Input(shape=input_shape)
x = Conv2D(kernel_size=3, filters=16, strides=1, padding='same', kernel_regularizer=l2(1e-4))(input)
x = Activation('elu')(x)

In [ ]:
#Stack 1
for block in range(2):
  y = Activation('elu')(x)
  y = Dropout(0.2)(y)
  y = Conv2D(kernel_size=3, filters=16, strides=1, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(y)
  y = Activation('elu')(x)
  y = Dropout(0.2)(y)
  y = Conv2D(kernel_size=3, filters=16, strides=1, padding='same', kernel_regularizer=l2(1e-4))(y)

  x = keras.layers.add([y, x])
  x = Activation('elu')(x)

#Stack 2
for block in range(2):
  y = Activation('elu')(x)
  y = Dropout(0.2)(y)
  y = Conv2D(kernel_size=3, filters=32, strides=1, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(y)
  y = Activation('elu')(x)
  y = Dropout(0.2)(y)
  y = Conv2D(kernel_size=3, filters=32, strides=1, padding='same', kernel_regularizer=l2(1e-4))(y)
  if block == 0:
    x_2 = Conv2D(kernel_size=1, filters=32, strides=1, padding='same')(x)
  else:
    x_2 = x
  x = keras.layers.add([y, x_2])
  x = Activation('elu')(x)

#Stack 3
for block in range(2):
  y = Activation('elu')(x)
  y = Dropout(0.2)(y)
  y = Conv2D(kernel_size=3, filters=64, strides=1, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(y)
  y = Activation('elu')(x)
  y = Dropout(0.2)(y)
  y = Conv2D(kernel_size=3, filters=64, strides=1, padding='same', kernel_regularizer=l2(1e-4))(y)
  if block == 0:
    x_2 = Conv2D(kernel_size=1, filters=64, strides=1, padding='same')(x)
  else:
    x_2 = x
  x = keras.layers.add([y, x_2])
  x = Activation('elu')(x)

#Stack 4
for block in range(2):
  y = Activation('elu')(x)
  y = Dropout(0.2)(y)
  y = Conv2D(kernel_size=3, filters=128, strides=1, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(y)
  y = Activation('elu')(x)
  y = Dropout(0.2)(y)
  y = Conv2D(kernel_size=3, filters=128, strides=1, padding='same', kernel_regularizer=l2(1e-4))(y)
  if block == 0:
    x_2 = Conv2D(kernel_size=1, filters=128, strides=1, padding='same')(x)
  else:
    x_2 = x
  x = keras.layers.add([y, x_2])
  x = Activation('elu')(x)

In [ ]:
x = Activation('elu')(x)
x = Dropout(0.2)(x)
x = AveragePooling2D(pool_size=8)(x)
output = Flatten()(x)
output = Dense(100, activation='softmax', kernel_initializer='he_normal')(output)

In [ ]:
model = Model(inputs=input, outputs=output)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 32, 32, 16)   0           activation[0][0]                 
_______________________________________________________________________________________

In [ ]:
opt = keras.optimizers.SGD(learning_rate=0.01, clipnorm=5)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=opt, metrics=["accuracy"])
callback = EarlyStopping(monitor='val_loss', patience=20)
history = model.fit(x_train, y_train, batch_size=128, epochs=200, verbose=1, validation_data=(x_test, y_test), callbacks=[callback])
model.save_weights(filepath=F"/content/gdrive/My Drive/Checkpoints/ResNet18_SGD_Dropout.h5")

Epoch 1/200
391/391 [==============================] - 21s 54ms/step - loss: 4.3233 - accuracy: 0.0543 - val_loss: 4.0409 - val_accuracy: 0.0910
Epoch 2/200
391/391 [==============================] - 20s 52ms/step - loss: 3.8230 - accuracy: 0.1299 - val_loss: 3.7991 - val_accuracy: 0.1358
Epoch 3/200
391/391 [==============================] - 20s 52ms/step - loss: 3.6032 - accuracy: 0.1701 - val_loss: 3.5521 - val_accuracy: 0.1826
Epoch 4/200
391/391 [==============================] - 20s 52ms/step - loss: 3.4612 - accuracy: 0.1923 - val_loss: 3.4781 - val_accuracy: 0.1937
Epoch 5/200
391/391 [==============================] - 20s 52ms/step - loss: 3.3583 - accuracy: 0.2118 - val_loss: 3.3568 - val_accuracy: 0.2205
Epoch 6/200
391/391 [==============================] - 20s 52ms/step - loss: 3.2505 - accuracy: 0.2346 - val_loss: 3.5180 - val_accuracy: 0.1973
Epoch 7/200
391/391 [==============================] - 20s 52ms/step - loss: 3.1534 - accuracy: 0.2522 - val_loss: 3.1889 - val_ac

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt2

history_dict = history.history
print(history_dict.keys())
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
train_acc = history_dict['accuracy']
test_acc = history_dict['val_accuracy']

epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


plt2.plot(epochs, train_acc, 'b', label='Training Accuracy')
plt2.plot(epochs, test_acc, color = 'red', label='Validation Accuracy')
plt2.title('Training and validation accuracy')
plt2.xlabel('Epochs')
plt2.ylabel('Accuracy')
plt2.legend()

plt2.show()

In [ ]:
y_pred = model.predict(x_test).argmax(-1)
import sklearn.metrics as metrics
print(metrics.classification_report(y_test.argmax(axis=1), y_pred))

              precision    recall  f1-score   support

           0       0.59      0.78      0.67       100
           1       0.63      0.54      0.58       100
           2       0.29      0.44      0.35       100
           3       0.22      0.21      0.22       100
           4       0.50      0.16      0.24       100
           5       0.52      0.34      0.41       100
           6       0.48      0.43      0.46       100
           7       0.65      0.46      0.54       100
           8       0.72      0.60      0.66       100
           9       0.54      0.59      0.56       100
          10       0.42      0.32      0.36       100
          11       0.28      0.32      0.30       100
          12       0.40      0.57      0.47       100
          13       0.39      0.62      0.48       100
          14       0.44      0.39      0.41       100
          15       0.46      0.39      0.42       100
          16       0.56      0.48      0.52       100
          17       0.66    

TESTS

In [ ]:
# model.load_weights(filepath=F"/content/gdrive/My Drive/Checkpoints/ResNet18_SGD_Dropout.h5")

model.load_weights("ResNet18_SGD_Dropout.h5")

In [ ]:
result = model.evaluate(batch_size=128, x=x_test, y=y_test)
dict(zip(model.metrics_names,result))

79/79 [==============================] - 1s 11ms/step - loss: 2.5824 - accuracy: 0.4836


{'accuracy': 0.483599990606308, 'loss': 2.582385301589966}